# Data Understanding
### a. Impor library dan Load Dataset

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
file_path = '/workspaces/codespaces-jupyter/data/processed/harga_sayuran_clean.csv'
data = pd.read_csv(file_path)

# Konversi 'Tanggal Harga' ke datetime, tangani format tanggal yang tidak konsisten
data['Tanggal Harga'] = pd.to_datetime(data['Tanggal Harga'], dayfirst=True, errors='coerce')

# Hapus baris dengan tanggal tidak valid
data = data.dropna(subset=['Tanggal Harga'])

# Ekstrak fitur dari tanggal
data['Year'] = data['Tanggal Harga'].dt.year
data['Month'] = data['Tanggal Harga'].dt.month

data.head()


### b. Visualisasi distribusi harga

In [ ]:
# Plot distribusi harga pasar untuk setiap jenis sayuran
plt.figure(figsize=(12, 6))
sns.boxplot(x='Jenis Sayuran', y='Harga Beli Pasar per Kilogram', data=data)
plt.title('Distribusi Harga Pasar untuk Setiap Jenis Sayuran')
plt.xticks(rotation=45)
plt.show()


### c. Visualisasi Tren Musiman

In [ ]:
# Plot rata-rata harga pasar per bulan untuk setiap jenis sayuran
plt.figure(figsize=(14, 7))
sns.lineplot(x='Month', y='Harga Beli Pasar per Kilogram', hue='Jenis Sayuran', data=data)
plt.title('Rata-rata Harga Pasar per Bulan untuk Setiap Jenis Sayuran')
plt.show()


### d. Visualisasi Tren Harga dari Waktu ke Waktu

In [ ]:
# Plot harga pasar dari waktu ke waktu untuk setiap jenis sayuran
plt.figure(figsize=(14, 7))
sns.lineplot(x='Tanggal Harga', y='Harga Beli Pasar per Kilogram', hue='Jenis Sayuran', data=data)
plt.title('Harga Pasar dari Waktu ke Waktu untuk Setiap Jenis Sayuran')
plt.xticks(rotation=45)
plt.show()


# Data Preparation
### a. Penambahan interaksi dan One-Hot Encode Variable Kategorikal

In [ ]:
# Buat interaksi antar fitur
data['Year_Month'] = data['Year'] * data['Month']

# One-hot encode variabel kategorikal
data_encoded = pd.get_dummies(data[['Year', 'Month', 'Year_Month', 'Jenis Sayuran', 'Season']])

# Gabungkan dengan variabel target
data_encoded['Harga Beli Pasar per Kilogram'] = data['Harga Beli Pasar per Kilogram']

# Pisahkan data menjadi set pelatihan dan pengujian
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_encoded, test_size=0.2, random_state=42)

# Pisahkan fitur dan variabel target
X_train = train_data.drop(columns=['Harga Beli Pasar per Kilogram'])
y_train = train_data['Harga Beli Pasar per Kilogram']
X_test = test_data.drop(columns=['Harga Beli Pasar per Kilogram'])
y_test = test_data['Harga Beli Pasar per Kilogram']

# Standarisasi fitur
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Modeling
### a. Model Latih SVR


In [ ]:
from sklearn.svm import SVR

# Inisialisasi model SVR
svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

# Latih model
svr.fit(X_train_scaled, y_train)

# Buat prediksi pada set pengujian
y_pred_svr = svr.predict(X_test_scaled)


b. Model latih Gradient Boosting sebagai pembanding

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Inisialisasi model GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Latih model
gb.fit(X_train_scaled, y_train)

# Buat prediksi pada set pengujian
y_pred_gb = gb.predict(X_test_scaled)


# Evaluation
### Hitung metrik evaluasi untuk kedua model

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Metrik evaluasi untuk SVR
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print(f"SVR - Mean Squared Error: {mse_svr}")
print(f"SVR - R-squared: {r2_svr}")

# Metrik evaluasi untuk Gradient Boosting
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f"Gradient Boosting - Mean Squared Error: {mse_gb}")
print(f"Gradient Boosting - R-squared: {r2_gb}")
